In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import __version__
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go 
init_notebook_mode(connected=True) 
import calendar
import os
import glob
import sys

In [ ]:
dist = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")

In [ ]:
dist.head()

In [ ]:
dist["district_id"]=dist["district_id"].astype("int32")

In [ ]:
import missingno as msgno
msgno.bar(dist,figsize=(5,2),sort="descending",color="powderblue",fontsize=20)

In [ ]:
def avgc(x):
    try:
        x_avg=sum(list(map(float,x.strip("[").split(", "))))/2
        return x_avg
    except:
        return x
dist["avg_pct_black/hispanic"] = dist["pct_black/hispanic"].apply(avgc) 
dist["avg_pct_free/reduced"] = dist["pct_free/reduced"].apply(avgc)
dist["avg_pp_total_raw"] = dist["pp_total_raw"].apply(avgc)
sns.heatmap(data=dist[["avg_pct_black/hispanic","avg_pct_free/reduced","avg_pp_total_raw"]].corr(),annot=True)

In [ ]:
def func(df,col1):
    fig,ax = plt.subplots(1,2,figsize=(10,5))
    sns.countplot(data=df,y=col1,ax=ax[0],order=df[col1].value_counts().index,orient="v")
    ax[0].set_title(col1)
    ax[1].pie(x=df[col1].value_counts(),labels=df[col1].value_counts().index,autopct='%1.0f%%')
    ax[1].set_title(col1)
print("Percentage of total number of districts having different pct_black/hisponic,pct_free/reduced,county_connections_ratio")
func(dist,"pct_black/hispanic")
func(dist,"pct_free/reduced")
func(dist,"county_connections_ratio")   

In [ ]:
print("Percentage of total number of districts having different pp_total_raw")
fig,ax = plt.subplots(1,1,figsize=(8,8))
explode = (0.01, 0.05, 0.05, 0.05,0.01,0.05,0.05,1,1,1,1)
colors= ("blue","red","green","yellow")
ax.pie(x=dist["pp_total_raw"].value_counts(),labels=dist["pp_total_raw"].value_counts().index,autopct='%1.1f%%',explode=explode,colors=colors)
ax.set_title("pp_total_raw")
dist[dist["pp_total_raw"]=="[32000, 34000["].style.applymap(lambda x: 'color:red;')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,5))
explode = (0.05, 0.05, 0.05, 0.05)
colors=(["#d45d00","#ff9100","#eaaa00","#6d6875"])
ax.pie(x=dist.locale.value_counts().values,explode=explode,labels=dist.locale.value_counts().index,autopct='%1.0f%%',colors=colors)
ax.set_title("dis_locale")

In [ ]:
def state(df,col1,col2,col3):
    pv=pd.pivot_table(data=df,index=[col3],aggfunc=[len],values=[col2],columns=[col1],fill_value=0)
    pv.plot.bar(figsize=(10,3)).legend(bbox_to_anchor=(1.02, 1),title=f"Total number of districts having different {col1}",fontsize='large')
state(dist,"pct_black/hispanic","district_id","state")
state(dist,"pct_free/reduced","district_id","state")
state(dist,"pp_total_raw","district_id","state")

In [ ]:
dist["state"].value_counts().index

In [ ]:
prod = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
prod.head()

In [ ]:
import missingno as msg
prod=prod.rename(columns={"LP ID":"lp_id"})
msg.bar(prod,figsize=(5,5),sort="descending",color="lightgreen",fontsize=8)

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(data=prod,y="Provider/Company Name",orient="v",order=prod["Provider/Company Name"].value_counts().head(14).index)
print("Name of Companies having maximum products")

In [ ]:
def function(x):
    try:
        if x[:2]=="LC":
            return "LC"
        elif x[:2]=="CM":
            return "CM"
        elif x[:3]=="SDO":
            return "SDO"
    except:
        return x
prod["Primary_essential_func"]=prod["Primary Essential Function"].apply(function)
fig,ax=plt.subplots(1,2,figsize=(8,8))
explode=(0.05,0.05,0.05)
ax[1].pie(x=prod["Primary_essential_func"].value_counts(),colors=["yellow","green","red"],labels=prod["Primary_essential_func"].value_counts().index,explode=explode,autopct="%1.0f%%")
sns.countplot(data=prod,x="Primary_essential_func",orient="v",ax=ax[0],palette="rainbow")

In [ ]:
plt.figure(figsize=(14,3))
sns.countplot(data=prod,x="Sector(s)",hue="Primary_essential_func",palette="rainbow")
plt.legend(loc=1)

In [ ]:
import glob 
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'
engagement_files = glob.glob(os.path.join(path, "*.csv"))
my_list = []
for i in engagement_files:
    my_file = pd.read_csv(i)
    filename = os.path.splitext(i)
    my_file['district_id'] = os.path.basename(filename[0])
    my_list.append(my_file)
    eng = pd.concat(my_list)

eng

In [ ]:
engagement_data_by_date1 = eng.groupby(by = 'time', as_index = False).agg('mean')

In [ ]:
figure = plt.figure(figsize = [15, 7])
sns.lineplot(x = 'time', y = 'engagement_index', data = engagement_data_by_date1, color = 'yellow')

plt.xlabel('Engagement Date', size = 14)
plt.ylabel('Engagement Index', size = 14)
plt.legend(labels = ['Engagement Index'], loc = 'upper left', frameon = False)
plt.title('Engagement Index Trend', size = 16)
plt.grid(b = True, axis = 'y')
plt.show()

In [ ]:
figure = plt.figure(figsize = [15, 7])
sns.lineplot(x = 'time', y = 'pct_access', data = engagement_data_by_date1, color = 'green')

plt.xlabel('Engagement Date', size = 14)
plt.ylabel('Percent Access', size = 14)
plt.legend(labels = ['Percent Access'], loc = 'upper left', frameon = False)
plt.title('Percent Access Trend', size = 16)
plt.grid(b = True, axis = 'y')
plt.show()

In [ ]:
engagement_data_by_date2 = eng.groupby(by = 'time', as_index = True).agg('mean')

In [ ]:
from pandas.plotting import lag_plot
plt.figure(figsize=(13,10))
lag_plot(engagement_data_by_date2.engagement_index)
plt.show()

In [ ]:
from pandas.plotting import lag_plot
plt.figure(figsize=(15,8))
lag_plot(engagement_data_by_date2.pct_access)
plt.show()